In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
import textwrap
from sklearn.metrics import classification_report
from pprint import pprint


import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertModel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
diagnoses_train = pd.read_csv("drive/MyDrive/MIDS/medal_mimic_subset/diagnoses/train.csv")
diagnoses_validation = pd.read_csv("drive/MyDrive/MIDS/medal_mimic_subset/diagnoses/valid.csv")
diagnoses_test = pd.read_csv("drive/MyDrive/MIDS/medal_mimic_subset/diagnoses/test.csv")

In [4]:
total_abbreviations = pd.read_csv("drive/MyDrive/MIDS/medal_mimic_subset/total_abbreviations.csv")

In [5]:
print(len(diagnoses_train))
print(len(diagnoses_validation))
print(len(diagnoses_test))

61079
9258
14287


In [6]:
# diagnoses_train.head()
# print(diagnoses_train[diagnoses_train.SUBJECT_ID == 29487].TEXT[0])
print(diagnoses_train[diagnoses_train.SUBJECT_ID == 29487].TEXT.values[-1])
# diagnoses_train.HOSPITAL_EXPIRE_FLAG.value_counts()

Chief Complaint : 
   24 Hour Events : 
    -  BP stable off pressors ,  no bolus requirement
    -  UOP adequate ,  still net positive
    -  tolerated 8 - 28 for 5 hrs ,  ABGs improved
    -  admin Lasix 40mg IV this am
   Pt intubated ,  sedated ,  opening eyes but only intermittently responding
   to commands . 
   Allergies : 
   Iodine
   Anaphylaxis ; 
   Cipro  ( Oral )   ( Ciprofloxacin Hcl ) 
   Rash ; 
   Sulfonamides
   Rash ; 
   Morphine
   Nausea / Vomiting
   Codeine
   Nausea / Vomiting
   Levofloxacin
   Anaphylaxis ; 
   Last dose of Antibiotics : 
   Vancomycin  -  2163 - 1 - 19 12 : 45 PM
   Piperacillin / Tazobactam  ( Zosyn )   -  2163 - 1 - 22 04 : 20 AM
   Infusions : 
   Fentanyl  -  25 mcg / hour
   Other ICU medications : 
   Other medications : 
   Flowsheet Data as of  2163 - 1 - 22 06 : 28 AM
   Vital signs
   Hemodynamic monitoring
   Fluid balance
                                                                  24 hours
                                

In [7]:
abbreviations = set(total_abbreviations.abbreviation)

def has_any_abbreviation(text):
    words = set(text.split())
    found = words.intersection(abbreviations)
    return bool(found)

diagnoses_train_subset = diagnoses_train.loc[diagnoses_train.TEXT.apply(has_any_abbreviation)].reset_index(drop=True)
diagnoses_validation_subset = diagnoses_validation.loc[diagnoses_validation.TEXT.apply(has_any_abbreviation)].reset_index(drop=True)
diagnoses_test_subset = diagnoses_test.loc[diagnoses_test.TEXT.apply(has_any_abbreviation)].reset_index(drop=True)

In [8]:
print(f"Train: {len(diagnoses_train_subset)} out of {len(diagnoses_train)}")
print(f"Validation: {len(diagnoses_validation_subset)} out of {len(diagnoses_validation)}")
print(f"Test: {len(diagnoses_test_subset)} out of {len(diagnoses_test)}")

Train: 60444 out of 61079
Validation: 9147 out of 9258
Test: 14132 out of 14287


In [20]:
with open("drive/MyDrive/MIDS/medal_mimic_subset/diagnoses/diag_to_idx.csv") as f:
    diagnosis_to_idx = {diag: int(idx) for diag, idx in csv.reader(f)}
    idx_to_diagnosis = {int(idx): diag for diag, idx in diagnosis_to_idx.items()}

print(f"{len(diagnosis_to_idx)} unique diagnosis codes")

1204 unique diagnosis codes


In [10]:
model_checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [31]:
MAX_LENGTH = 512


def head_and_tail(tokens, total_size=MAX_LENGTH):
    """Build an array of size <total_size> using the head and tail ends of the input array"""
    half = int(total_size / 2)
    if len(tokens) <= total_size:
        return tokens
    head = tokens[:half]
    tail = tokens[-half:]
    return np.concatenate([head, tail])


def tokenize(texts):
    """
    Tokenize an array of text inputs for bert, and take the head and tail of
    each set of bert inputs. Returns a tuple of three arrays: input_ids, token_type_ids,
    and attention_mask.
    """
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []

    for text in texts:
        bert_output = bert_tokenizer(
            text, padding="max_length", return_tensors="tf", max_length=MAX_LENGTH
        )
        input_ids = bert_output["input_ids"][0]
        token_type_ids = bert_output["token_type_ids"][0]
        attention_mask = bert_output["attention_mask"][0]

        input_ids_list.append(head_and_tail(input_ids))
        token_type_ids_list.append(head_and_tail(token_type_ids))
        attention_mask_list.append(head_and_tail(attention_mask))

    return np.array(input_ids_list), np.array(token_type_ids_list), np.array(attention_mask_list)


def get_labels(df):
    """
    Get array of labels for each record in the dataframe as a one-hot encoded array, e.g.
    each ICD9 code appears as a 0 or 1 mapping to the diagnosis_labels array. If the ICD9
    code does not appear in the training set, it is ignored.
    """
    labels_by_hadm_id = {row.HADM_ID: row.ICD9_ID for _, row in df.iterrows()}
    output = np.zeros((len(df), len(diagnosis_to_idx)))
    for sample_idx, row in enumerate(df.iterrows()):
        for icd9 in row[1].ICD9_ID.split(';'):
            # skip codes that are not present in the train set
            grouped_icd9 = icd9[:4] if icd9.startswith("V") or icd9.startswith("E") else icd9[:3]
            if (index := diagnosis_to_idx.get(grouped_icd9)) is None:
                continue

            output[sample_idx, index] = 1

    return output


def preprocess_data(batch_df):
    return tokenize(batch_df.TEXT), get_labels(batch_df)


input_examples, label_examples = preprocess_data(diagnoses_train_subset[:3])
print("Labels:", [idx_to_diagnosis[idx] for idx in np.where(label_examples[1] == 1)[0]])
print("Inputs:", bert_tokenizer.decode(input_examples[0][1]))

Labels: ['584', '707', '428', '276', '403', '293', '585', '427', '530', '197', '486', '038', '785', '995', '518', '599', '424', 'V104', 'V105', '274', 'V436']
Inputs: [CLS] Chief Complaint : 24 Hour Events : EKG - At 2163 - 1 - 17 11 : 30 AM - Afib, RBBB, no acute ST - T waves changes ARTERIAL LINE - START 2163 - 1 - 17 04 : 15 PM - CE negative - vanc added - urine grew out pseudomonas, did not double cover per ID - MRI hip ordered, not performed - uable to wean pressors overnight Pt doing okay this am, still some L hip pain, but otherwise, breathing comfortably, no CP / SOB / Abd pain. Allergies : Iodine Anaphylaxis ; Cipro ( Oral ) ( Ciprofloxacin Hcl ) Rash ; Sulfonamides Rash ; Morphine Nausea / Vomiting Codeine Nausea / Vomiting Levofloxacin Anaphylaxis ; Last dose of Antibiotics : Vancomycin - 2163 - 1 - 17 09 : 45 AM Piperacillin / Tazobactam ( Zosyn ) - 2163 - 1 - 18 04 : 12 AM Infusions : Other ICU medications : Heparin, monitor for worsening CHF URINARY TRACT INFECTION ( UTI 

In [35]:
BATCH_SIZE = 32


class DataGeneratorFromDataframe(tf.keras.utils.Sequence):

    def __init__(self, df, batch_size=BATCH_SIZE, shuffle=True):
        self.df = df
        self.n_examples = len(df)
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(self.n_examples)
        self.on_epoch_end()

    def __len__(self):
        # Return the number of batches in the full dataset
        return self.n_examples // self.batch_size

    def __getitem__(self, idx):
        batch_start = idx * self.batch_size
        batch_end = (idx + 1) * self.batch_size

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this batch
        batch_idx_skip = self.row_order[:batch_start] + self.row_order[batch_end:]
        batch_df = self.df.loc[batch_start : batch_end].copy()
        batch_data = preprocess_data(batch_df)

        return batch_data

    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [37]:
# Create an instance of our data generator, for our training data file and size

train_data_generator = DataGeneratorFromDataframe(diagnoses_train_subset)
valid_data_generator = DataGeneratorFromDataframe(diagnoses_validation_subset)
test_data_generator = DataGeneratorFromDataframe(diagnoses_test_subset)

In [40]:
def create_diagnosis_bert_model(
    model_checkpoint=model_checkpoint,
    n_classes=len(diagnosis_to_idx),
    hidden_size=201,
    dropout=0.3,
    learning_rate=0.00005,
):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes.
    """
    keras.backend.clear_session()

    input_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_model = TFBertModel.from_pretrained(model_checkpoint)
    bert_out = bert_model(bert_inputs)

    pooler_token = bert_out[1]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    classification = tf.keras.layers.Dense(n_classes, activation='sigmoid', name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics='accuracy')

    return classification_model

In [41]:
diagnosis_bert_model = create_diagnosis_bert_model()
diagnosis_bert_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 512)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 512)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 512)]                0         []                            
 tLayer)                                                                                      

In [42]:
checkpoint_dir = 'drive/MyDrive/MIDS/model_checkpoints/diagnosis_base'
checkpoint_filepath = checkpoint_dir + 'weights.{epoch:02d}-{val_accuracy:.2f}.model.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
)

diagnosis_history = diagnosis_bert_model.fit(
    train_data_generator,
    validation_data=valid_data_generator,
    batch_size=BATCH_SIZE,
    epochs=5,
    callbacks=[model_checkpoint_callback],
)

Epoch 1/5
1888/1888 [==============================] - ETA: 0s - loss: 0.0844 - accuracy: 0.0387

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


1888/1888 [==============================] - 1794s 921ms/step - loss: 0.0844 - accuracy: 0.0387 - val_loss: 0.0468 - val_accuracy: 0.0330
Epoch 2/5
1888/1888 [==============================] - 1726s 914ms/step - loss: 0.0478 - accuracy: 0.0766 - val_loss: 0.0445 - val_accuracy: 0.1610
Epoch 3/5
1888/1888 [==============================] - 1726s 914ms/step - loss: 0.0422 - accuracy: 0.1098 - val_loss: 0.0421 - val_accuracy: 0.1357
Epoch 4/5
1888/1888 [==============================] - 1728s 915ms/step - loss: 0.0371 - accuracy: 0.1148 - val_loss: 0.0404 - val_accuracy: 0.1357
Epoch 5/5
1888/1888 [==============================] - 1726s 914ms/step - loss: 0.0331 - accuracy: 0.1150 - val_loss: 0.0388 - val_accuracy: 0.1079


In [44]:
def compute_top_k_recall(labels, predictions, k=10):
    # Get indices of top-k predictions
    idxs = np.argsort(predictions, axis=1)[:, ::-1][:, :k]

    # Gather top-k labels
    top_k_labels = labels[np.arange(labels.shape[0])[:, np.newaxis], idxs]

    # Compute recall for each sample
    sum_top_k_labels = np.sum(top_k_labels, axis=1)
    sum_labels = np.sum(labels, axis=1)
    recall_per_sample = sum_top_k_labels / sum_labels

    # Compute mean recall across all samples
    return np.mean(recall_per_sample)


def evaluate(model, test_data):
  for inputs, labels in test_data:
    predictions = model.predict(inputs)
    return {
        "top_10_recall": compute_top_k_recall(labels, predictions),
        "top_5_recall": compute_top_k_recall(labels, predictions, k=5),
        "top_30_recall": compute_top_k_recall(labels, predictions, k=30)
    }

evaluate(diagnosis_bert_model, test_data_generator)

TypeError: evaluate() missing 2 required positional arguments: 'inputs' and 'labels'